# Image Inpainting

## Traditional Methods

OpenCV implements two traditional methods for inpainting:

* cv2.INPAINT_TELEA: An image inpainting technique based on the fast marching method (Telea, 2004)

* cv2.INPAINT_NS: Navier-stokes, Fluid dynamics, and image and video inpainting (Bertalmío et al., 2001)

In [1]:
import cv2
import matplotlib.pyplot as plt

# read image
img = cv2.imread('cat_damaged.png')
img_toplot = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# plot image
plt.imshow(img_toplot)
plt.axis('off')
plt.show()

# read mask
mask = cv2.imread('cat_mask.png', 0)

# plot mask
plt.imshow(mask, cmap='gray')
plt.axis('off')

error: OpenCV(4.8.0) /io/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [ ]:
# inpaint using OpenCV
dst = cv2.inpaint(img, mask, 3, cv2.INPAINT_NS)
cv2.imshow(dst)

DisabledFunctionError: cv2.imshow() is disabled in Colab, because it causes Jupyter sessions
to crash; see https://github.com/jupyter/notebook/issues/3935.
As a substitution, consider using
  from google.colab.patches import cv2_imshow


---
## Deep Learning Methods

Generative Image Inpainting with Contextual Attention (Yu et al. 2018)

Adapted from: https://github.com/daa233/generative-inpainting-pytorch


In [6]:
!unzip model.zip

Archive:  model.zip
error [model.zip]:  missing 9943 bytes in zipfile
  (attempting to process anyway)
error: invalid zip file with overlapped components (possible zip bomb)


In [7]:
import os
import random

import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.utils as vutils
import matplotlib.pyplot as plt

from PIL import Image

from model.networks import Generator
from utils.tools import normalize, get_model_list

checkpoint_path = 'model/pretrained'
output_path = 'output.png'
image_shape = [256, 256, 3]
netG_config = {'input_dim': 3, 'ngf': 32}
cuda = False
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ModuleNotFoundError: No module named 'model'

In [ ]:
x = Image.open('test.png')
mask = Image.open('test_mask.png')
x

In [ ]:
transform = transforms.Compose([
            transforms.Resize(image_shape[:-1]),
            transforms.CenterCrop(image_shape[:-1]),
            transforms.ToTensor(),
])

x = transform(x)
mask = transform(mask)
mask = mask[0].unsqueeze(dim=0)

x_show = x.clone()
x = normalize(x)
x = x * (1. - mask)
x = x.unsqueeze(dim=0)
mask = mask.unsqueeze(dim=0)

In [4]:
with torch.no_grad():
  # build network
  netG = Generator(netG_config, False, device)
  last_model_name = get_model_list(checkpoint_path, "gen")
  print(last_model_name)

  # load pretrained weights
  ckpt = torch.load(last_model_name)
  netG.load_state_dict(ckpt)

  # do inference
  _, x2, _ = netG(x, mask)
  #x2 is a tensor with shape

  # save and visualize result
  #print(x2)
  print(x2.shape)

  vutils.save_image(x2, 'x2.png', padding=0, nrmalize=True)
  im= Image.open('x2.png')
  im

  #better image for sure than traditional methods, still a bit fuzzy when zooming in (dependent on amount of images the model can train on)

NameError: name 'torch' is not defined